<a href="https://colab.research.google.com/github/danielbauer1860/LDS_Project/blob/main/proposal/generating_news_dataset_proposal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This notebook references [the project of Salminen et al. (2021)](https://github.com/joolsa/FakeReviews/blob/)

In [ ]:
!pip install iteround -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Loading the model

Redownloading the model required some tweaks, which are described in:
https://stackoverflow.com/questions/76459034/how-to-load-a-fine-tuned-peft-lora-model-based-on-llama-with-huggingface-transfo

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import infer_auto_device_map, init_empty_weights

peft_model_id = 'dbauer1860/llama-2-bnc-baby-news'

config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype='auto',
    device_map='auto',
    offload_folder="offload", offload_state_dict = True
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

# Extracting Word Counts and Prompts

In [ ]:
import pandas as pd
import numpy as np
import random
import nltk
from iteround import saferound
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Linguistic Data Science/data/bnc_baby_texts.csv', sep='|')

In [ ]:
df.head()

,text,category
0,Oxford Art Journal. Sample containing about ...,ACA
1,The Lancet. Sample containing about 44333 wo...,ACA
2,Computers and the humanities. Sample contain...,ACA
3,British journal of social work. Sample conta...,ACA
4,The British polity. Sample containing about ...,ACA


In [ ]:
def split_into_sentences(text):
    return nltk.sent_tokenize(text)

In [ ]:
df['sentences'] = df['text'].apply(split_into_sentences)

In [ ]:
df

,text,category,sentences
0,Oxford Art Journal. Sample containing about ...,ACA,"[ Oxford Art Journal., Sample containing abou..."
1,The Lancet. Sample containing about 44333 wo...,ACA,"[ The Lancet., Sample containing about 44333 ..."
2,Computers and the humanities. Sample contain...,ACA,"[ Computers and the humanities., Sample conta..."
3,British journal of social work. Sample conta...,ACA,"[ British journal of social work., Sample con..."
4,The British polity. Sample containing about ...,ACA,"[ The British polity., Sample containing abou..."
...,...,...,...
177,Liverpool Daily Post and Echo: Arts section....,NEWS,[ Liverpool Daily Post and Echo: Arts section...
178,"The Guardian, electronic edition of 1989-12-...",NEWS,"[ The Guardian, electronic edition of 1989-12..."
179,"Independent, electronic edition of 1989-10-0...",NEWS,"[ Independent, electronic edition of 1989-10-..."
180,The Scotsman: Applied Science pages. Sample ...,NEWS,"[ The Scotsman: Applied Science pages., Sampl..."


In [ ]:
def get_random_sentence(sentences_list, random_state=1860, min_word_count=5):
    random.seed(random_state)
    filtered_sentences = [sentence for sentence in sentences_list if len(word_tokenize(sentence)) > min_word_count]
    return random.choice(filtered_sentences) if filtered_sentences else None

In [ ]:
df['prompt'] = df['sentences'].apply(get_random_sentence)

In [ ]:
def get_samples(df, n=10, state=1860):
  return df.sample(n=n, random_state=state)

In [ ]:
df_aca, df_dem, df_fic, df_news = [get_samples(y) for x, y in df.groupby(['category'])]

<ipython-input-15-c993aa370679>:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  df_aca, df_dem, df_fic, df_news = [get_samples(y) for x, y in df.groupby(['category'])]


In [ ]:
df_news

,text,category,sentences,prompt
93,"The Guardian, electronic edition of 1989-12-...",NEWS,"[ The Guardian, electronic edition of 1989-12...",’ \n The Joint Command began taking risks .
179,"Independent, electronic edition of 1989-10-0...",NEWS,"[ Independent, electronic edition of 1989-10-...",It has become increasingly apparent that ...
96,"Independent, electronic edition of 1989-10-0...",NEWS,"[ Independent, electronic edition of 1989-10-...",If the builder is employed by an archit...
99,Liverpool Daily Post and Echo: Applied Scien...,NEWS,[ Liverpool Daily Post and Echo: Applied Scie...,"’ \n In the meantime , the tendering proc..."
181,Liverpool Daily Post and Echo: Leisure pages...,NEWS,[ Liverpool Daily Post and Echo: Leisure page...,I 'll fight on — Foreman \n GEORGE FOREMA...
159,"Independent, electronic edition of 1989-10-1...",NEWS,"[ Independent, electronic edition of 1989-10-...",The awkward changes of tone caused by t...
94,"Independent, electronic edition of 1989-10-0...",NEWS,"[ Independent, electronic edition of 1989-10-...",Keeping the fabric of history : Taking c...
171,"Independent, electronic edition of 1989-10-0...",NEWS,"[ Independent, electronic edition of 1989-10-...",Sculpted from white plaster like the stu...
136,"The Guardian, electronic edition of 1989-11-...",NEWS,"[ The Guardian, electronic edition of 1989-11...","’ \n With the backing of the BBC , the ..."
108,Northern Echo: Arts section. Sample containi...,NEWS,"[ Northern Echo: Arts section., Sample contai...",In the finale the abrupt changes of pac...


In [ ]:
aca_prompts = df_aca['prompt'].to_list()
fic_prompts = df_fic['prompt'].to_list()
news_prompts = df_news['prompt'].to_list()

# Generating output

In [ ]:
#Generation parameters; these were, for the most part, set in accordance to Salminen et al.
#the only difference is the implementation of a deviation value
do_sample = True
top_k = 0
top_p = 0.92
temperature = 0.7
eos_token = model.config.eos_token_id
device = "cuda:0"
deviation = 5

In [ ]:
#from Salminen et al.; a seed value was set to allow the generation to be re-produced and continued over multiple sessions
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

The generating loop

In [ ]:
def generate(prompts):
  outputs = []
  for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    random_seed(seed_value=1860, use_cuda=True)
    o = model.generate(**inputs, do_sample=do_sample, top_k=top_k, top_p=top_p, temperature=temperature, eos_token_id=eos_token, max_length=500)
    outputs.append(tokenizer.decode(o[0], skip_special_tokens=True))
  return outputs

In [ ]:
aca_outputs = generate(aca_prompts)

In [ ]:
fic_outputs = generate(fic_prompts)

In [ ]:
news_outputs = generate(news_prompts)

In [ ]:
aca_results = pd.DataFrame(aca_outputs, columns=['output'])
aca_results['prompt_category'] = 'ACA'
aca_results['prompt'] = aca_prompts

fic_results = pd.DataFrame(fic_outputs, columns=['output'])
fic_results['prompt_category'] = 'FIC'
fic_results['prompt'] = fic_prompts

news_results = pd.DataFrame(news_outputs, columns=['output'])
news_results['prompt_category'] = 'NEWS'
news_results['prompt'] = news_prompts

result_frames = [aca_results, fic_results, news_results]

result_df = pd.concat(result_frames, ignore_index=True)
result_df

,output,prompt_category,prompt
0,These conditions typically exist in garag...,ACA,These conditions typically exist in garag...
1,The irony of this lay in the fact that...,ACA,The irony of this lay in the fact that...
2,I arranged to visit that afternoon and ...,ACA,I arranged to visit that afternoon and ...
3,The dissemination of information in the ...,ACA,The dissemination of information in the ...
4,Even if a species somehow managed to be...,ACA,Even if a species somehow managed to be...
5,"The woody monocotyledon , Cordylinefruticos...",ACA,"The woody monocotyledon , Cordylinefruticos..."
6,In the cytogenetic study only 3 metaphas...,ACA,In the cytogenetic study only 3 metaphas...
7,"The posture of many dinosaurs , and part...",ACA,"The posture of many dinosaurs , and part..."
8,At a recent meeting of the North West ...,ACA,At a recent meeting of the North West ...
9,10 Conclusion \n I have stressed that I ...,ACA,10 Conclusion \n I have stressed that I ...


In [ ]:
result_df.to_csv('/content/drive/MyDrive/Linguistic Data Science/data/llama-2-news-output.csv', index=False)